In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Feb  2 13:04:21 2019

@author:
"""
import math
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from matplotlib import pyplot
import pandas
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

NUMERICALS = ['GIAGE1', 'HA_HEIGHT', 'HA_WEIGHT', 'TUDRPRWK', 'B1FND', 'HA_SMOKE_0.0', 'HA_SMOKE_1.0', 'HA_SMOKE_2.0', 'GIERACE_1.0',
              'GIERACE_2.0', 'GIERACE_3.0', 'GIERACE_4.0', 'GIERACE_5.0', 'CLINIC_1.0','CLINIC_2.0', 'CLINIC_3.0',  
              'CLINIC_4.0', 'CLINIC_5.0', 'CLINIC_6.0','NFWLKSPD_0.0', 'NFWLKSPD_1.0', 'NFWLKSPD_2.0', 'BUAMEAN', 'QUIMEAN', 'SOSMEAN', 'PHYS_MROS_1.0', 'PHYS_MROS_2.0', 'PHYS_MROS_3.0']


# change allele to number
def allele_to_number(sample, attribute, sig_dict):
    return sample[attribute].count(sig_dict[attribute])


# fill numerical empty cells with median of the column, race with 1 (for white) and other categorical empty cells with the 0
def fill_empty_cell(sample, attribute, data):
    if pandas.isnull(sample[attribute]):
        if attribute in NUMERICALS:
            return data[attribute].median()
        return int(attribute == 'GIERACE' )
    elif attribute == 'FRAC':
        return int(sample['FAANYSLD'] or sample['FAANYWST'] or sample['FAANYHIP'] or sample['XMDSQGE1'])
    return sample[attribute]


# calculate GRS
def load_weight(sheet):
    weight = []
    df = pandas.read_excel('Estrada_63.xlsx', sheet_name=sheet)
    w = list(df)[-1]
    for i in range(len(df)):
        weight.append(df.iloc[i][w])
    return weight

if __name__ == '__main__':
    data = pandas.read_excel('mros_1103snps.xlsx')
    # drop HA_SLDFXFU where only 10% is filled, drop subjectid,
    data.drop(['HA_SLDFXFU', 'TURSMOKE', 'HA_SLDFX', 'HA_WRSTFX'], axis=1, inplace=True)
    # make the fractures into 1 variable
    data['FRAC'] = 0
    for attribute in data.keys():
        data[attribute] = data.apply(lambda sample: fill_empty_cell(sample, attribute, data), axis=1)
    # drop the other fractured values
    data.drop(['FAANYSLD', 'FAANYWST'], axis=1, inplace=True)
    # encode the categorical data
#     data = pandas.DataFrame(pandas.get_dummies(data, columns=['GIERACE', 'CLINIC', 'NFWLKSPD', 'HA_SMOKE', 'PHYS_MROS']))
    # features = list(data)[22:-6]
    # setting Y and X
    # Y_df = np.asarray(data['B1THD'], dtype="|S8")
    Y_df = data['FRAC']
    X_df = pandas.read_excel('ready_to_go.xlsx')
    #X_df = data.drop(['SUBJECTID', 'HA_LSD', 'BUAMEAN', 'FAHIPFV1', 'FASLDFV1', 'FAWSTFV1', 'EFSTATUS',
    #                    'XMDSQGE1', 'XMSQGE2',  'HA_BMI', 'FAANYHIP', 'HA_CALCIUM', 'FRAC', 'PHYS_MROS'], axis=1)
    # weight_LS = load_weight('LS_sex-combined_beta')
   # feature_data = data[features]
    # weight_LS = pandas.DataFrame(pandas.Series(weight_LS, index=features, name=0))
    #weight_FN = load_weight('FN_sex-combined_beta')
    # X_df['GRS_LS'] = feature_data.dot(weight_LS)
   # X_df['GRS_FN'] = feature_data.dot(weight_FN)
   # X_df.drop(features, axis=1, inplace=True)
    # print(Y_df.shape)
    
    # to export the fracture and subject ID
#     export = data[['SUBJECTID', 'FRAC', 'SOSMEAN', 'B1FND', 'B1THD', 'B1TLD', 'W_SCORE', 'GIAGE1', 'HA_HEIGHT', 'HA_WEIGHT', 'TUDRPRWK', 'QLFXST51', 'NFWLKSPD', 'PHYS_MROS', 'GIERACE', 'HA_SMOKE', 'CLINIC']]
#     export.to_csv('export.csv')
#     data.to_csv('X_variables.csv')
    
#     Y_df.to_csv('MOF.csv')
    
    print(Y_df.head())
    print(X_df[1:1])
    print(X_df.shape)
    print(Y_df.shape)

    with open('datamros1103', 'wb') as data_file_handler:
        import pickle

        pickle.dump(
            dict(
                X=X_df,
                Y=Y_df
            ),
            data_file_handler
        )


In [ ]:
import pandas as pd

data = pd.read_excel('mros_1103snps.xlsx')

data.isnull().sum()

data1 = data.fillna(" ")

data1.insull().sum()

In [ ]:
import numpy as np

Y_df.value_counts()




In [ ]:
import numpy
import pandas

cv = pandas.read_excel('contious_variable.xlsx')
    
mean_cv = numpy.mean(cv, axis=0)
std_cv = numpy.std(cv, axis=0)
stan_cv = (cv - mean_cv) / std_cv  # mean variance normalization
    
stan_cv.to_csv('norma_continu_var.csv')

In [ ]:
# Logistic regression

import pickle
import pandas as pd

with open('datamros1103', 'rb') as file_handler:
    data = pickle.load(file_handler)
    X, Y = data.get('X', []).values, data.get('Y', []).values

import numpy as np
import tensorflow as tf
import random as rn

import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE

# fix random seed for reproducibility
seed = 6
#
# # The below is necessary for starting Numpy generated random numbers
# # in a well-defined initial state.
np.random.seed(seed)
# # The below is necessary for starting core Python generated random numbers
# # in a well-defined state.
rn.seed(seed)

# according to keras documentation, numpy seed should be set before importing keras
# information regarding setup for obtaining reproducible results using Keras during development in the following link
# https://keras.io/getting-started/faq/#how-can-i-obtain-reproducible-results-using-keras-during-development
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import label_binarize
from keras.utils import to_categorical

# The below tf.set_random_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
#tf.set_random_seed(seed)
# Y = label_binarize(Y, classes=[0,1])

batch_size = 100
num_classes = 2

optimizer = 'adamax'
from keras import optimizers


number_of_data = X.shape[0]
number_of_train_data = int(.8*number_of_data)
number_of_test_data = number_of_data-number_of_train_data

# load dataset
x_train, x_test = X[:number_of_train_data, :], X[number_of_train_data:, :]
#mean_train_data = np.mean(train_data, axis=0)
#std_train_data = np.std(train_data, axis=0)
#x_train = (train_data - mean_train_data) / std_train_data  # mean variance normalization
#x_test = (test_data - mean_train_data) / std_train_data  # mean variance normalization
y_train, y_test = Y[:number_of_train_data], Y[number_of_train_data:]
#x_test, y_test = sm.fit_resample(X,Y)

# X_df, Xtest, Y_df, ytest = train_test_split(X_df, Y_df, test_size=0.2)
# Xtrain, ytrain = sm.fit_resample(X_df, Y_df)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

y_train = y_train.astype('int32')
y_test = y_test.astype('int32')
# y_train = to_categorical(y_train)
#y_test = to_categorical(y_test)
from keras.regularizers import L1L2
from sklearn.linear_model import LogisticRegression
# def create_model():
#     model = LogisticRegression(solver='lbfgs',max_iter=35)
#     return model;
def create_model():
    model = Sequential()
    model.add(Dense(1, input_dim=26, kernel_initializer='normal', activation='sigmoid'))
    model.compile(optimizer='sgd',loss='binary_crossentropy', metrics=['accuracy'])
    return model

from sklearn.metrics import roc_curve
from sklearn.metrics import auc, roc_auc_score, precision_recall_curve
from sklearn.metrics import confusion_matrix, brier_score_loss, precision_score, recall_score, f1_score, average_precision_score, precision_recall_curve, accuracy_score
from sklearn import metrics
from inspect import signature

model = create_model()


sm = SMOTE(random_state=2, ratio = 1.0)
x_train_s, y_train_s = sm.fit_resample(x_train, y_train)

model.fit(x_train_s, y_train_s, epochs=1, batch_size=batch_size, verbose=1)
y_pred = model.predict(x_test)
print('The predicted y_value:', y_pred)
# y_pred = model.predict(x_test)
#y_pred = y_pred.astype('int32')
#print(y_pred, ' YPOST')
yscore_raw = model.predict_proba(x_test)
yscore = [s[0] for s in yscore_raw]
fpr, tpr, thresholds = roc_curve(y_test, yscore)
#final = [(lambda i: 0 if i <= np.average(thresholds) else 1)(i) for i in y_pred]
final = [(lambda i: 0 if i <= np.average(thresholds) else 1)(i) for i in y_pred]

print(confusion_matrix(y_test, final))

y_test = pd.Series(y_test)
y_test.to_csv('y_test.csv')

y_pred = pd.Series(final)
y_pred.to_csv('y_pred_frac_logi.csv')

# np.savetxt('logi_yscore_raw.csv', yscore_raw, delimiter=',')

# predict probabilities
# probs = model.predict_proba(x_test)

# keep probabilities for the positive outcome only
# probs = probs[:, 1]

# predict class values
# yhat = model.predict(x_test)

# calculate precision-recall AUC
# precision, recall, thresholds = precision_recall_curve(y_test, probs)

# calculate F1 score
# f1 = f1_score(y_test, yhat, normalize = False)

# calculate precision-recall AUC
# auc = auc(recall, precision)

# calculate average precision score

# ap = average_precision_score(y_test, probs)
# print('f1=%.3f auc=%.3f' % (f1, auc, ap))

# plot no skill
# plt.plot([0, 1], [0.5, 0.5], linestyle='--')

# plot the precision-recall curve for the model
# plt.plot(recall, precision, marker='.')

# show the plot
# plt.show()


#print(list(X))
#print("\tBrier: %1.3f" % (clf_score))
#print("\tPrecision: %1.3f" % precision_score(y_test, y_pred))
#print("\tRecall: %1.3f" % recall_score(y_test, y_pred))
#print("\tF1: %1.3f\n" % f1_score(y_test, y_pred))

# average_precision = average_precision_score(y_test, yscore)

# print('Average precision-recall score: {0:0.2f}'.format(average_precision))
# precision, recall, _ = precision_recall_curve(y_test, yscore)

#step_kwargs = ({'step': 'post'}
#              if 'step' in signature(plt.fill_between).parameters
#              else {})

# plt.step(recall, precision, color='b', alpha=0.2, where='post')
# plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)

# plt.xlabel('Recall')
# plt.ylabel('Precision')
# plt.ylim([0.0, 1.05])
# plt.xlim([0.0, 1.0])
# plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(average_precision))
# plt.savefig('logistic_Regression_PRCurve.png')
# plt.show()   # Display

# y_test =  pd.Series(y_test)
# y_test.to_csv('y_test.csv')

# logi_final = pd.Series(final)
# logi_final.to_csv('logi_y_pred.csv')

# Calculate the 95% CI for AUC in Logistic Regression
def conf_auc(test_predictions, ground_truth, bootstrap=1000, seed=None,  confint=0.95):
    """Takes as input test predictions, ground truth, number of bootstraps, seed, and confidence interval"""
    #inspired by https://stackoverflow.com/questions/19124239/scikit-learn-roc-curve-with-confidence-intervals by ogrisel
    bootstrapped_scores = []
    rng = np.random.RandomState(seed)
    if confint>1:
        confint=confint/100
    for i in range(bootstrap):
        # bootstrap by sampling with replacement on the prediction indices
        indices = rng.randint(0, len(test_predictions) - 1, len(test_predictions))
        if len(np.unique(ground_truth[indices])) < 2:
            continue

        score = metrics.roc_auc_score(ground_truth[indices], test_predictions[indices])
        bootstrapped_scores.append(score)
    
    sorted_scores = np.array(bootstrapped_scores)
    sorted_scores.sort()

    lower_bound=(1-confint)/2
    upper_bound=1-lower_bound
    confidence_lower = sorted_scores[int(lower_bound * len(sorted_scores))]
    confidence_upper = sorted_scores[int(upper_bound * len(sorted_scores))]
    auc = metrics.roc_auc_score(ground_truth, test_predictions)
    print("{:0.0f}% confidence interval for the score: [{:0.3f} - {:0.3}] and your AUC is: {:0.3f}".format(confint*100, confidence_lower, confidence_upper, auc))
    confidence_interval = (confidence_lower, auc, confidence_upper)
    return confidence_interval

conf_auc(y_pred, y_test)


# Calculate the 95% CI for Accuracy in Logistic Regression
def conf_accu(test_predictions, ground_truth, bootstrap=1000, seed=None,  confint=0.95):
    """Takes as input test predictions, ground truth, number of bootstraps, seed, and confidence interval"""
    #inspired by https://stackoverflow.com/questions/19124239/scikit-learn-roc-curve-with-confidence-intervals by ogrisel
    bootstrapped_scores = []
    rng = np.random.RandomState(seed)
    if confint>1:
        confint=confint/100
    for i in range(bootstrap):
        # bootstrap by sampling with replacement on the prediction indices
        indices = rng.randint(0, len(test_predictions) - 1, len(test_predictions))
        if len(np.unique(ground_truth[indices])) < 2:
            continue
        
        score = metrics.accuracy_score(ground_truth[indices], test_predictions[indices], normalize=False)
        bootstrapped_scores.append(score)
    
    sorted_scores = np.array(bootstrapped_scores)
    sorted_scores.sort()

    lower_bound=(1-confint)/2
    upper_bound=1-lower_bound
    confidence_lower = sorted_scores[int(lower_bound * len(sorted_scores))]
    confidence_upper = sorted_scores[int(upper_bound * len(sorted_scores))]
    auc = metrics.accuracy_score(ground_truth, test_predictions.round(indices), normalize=False)
    print("{:0.0f}% confidence interval for the score: [{:0.3f} - {:0.3}] and your accuracy is: {:0.3f}".format(confint*100, confidence_lower, confidence_upper, auc))
    confidence_interval = (confidence_lower, auc, confidence_upper)
    return confidence_interval

#conf_accu(y_pred, y_test)

auc = metrics.roc_auc_score(y_test, yscore)
plt.plot(fpr, tpr, '^k:', label='%s ROC (area = %0.2f)' % ('Logistic Regression', auc))

# Calculate Area under the curve to display on the plot

# Now, plot the computed values

# Custom settings for the plot
# Receiver Operating Characteristic-
plt.plot([0, 1], [0, 1], '-k')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1-Specificity(False Positive Rate)')
plt.ylabel('Sensitivity(True Positive Rate)')
plt.title('Logistic Regression')
plt.legend(loc="lower right")
plt.savefig('logistic_Regression_MOF.png')
plt.show()   # Display


In [ ]:
# Random forest

import itertools
import time

import numpy
import pandas
import scikitplot
from imblearn.over_sampling import SMOTE
from matplotlib import pyplot
from scipy import stats
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve
from sklearn.metrics import auc, roc_auc_score, precision_recall_curve
from sklearn.metrics import confusion_matrix, brier_score_loss, precision_score, recall_score, f1_score, average_precision_score, precision_recall_curve, accuracy_score
from sklearn import metrics
from sklearn.model_selection import KFold  # http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html#sklearn.model_selection.StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split

NUMERICALS = ['GIAGE1', 'HA_HEIGHT', 'HA_WEIGHT', 'TUDRPRWK', 'B1FND', 'HA_SMOKE_0.0', 'HA_SMOKE_1.0', 'HA_SMOKE_2.0', 'GIERACE_1.0',
              'GIERACE_2.0', 'GIERACE_3.0', 'GIERACE_4.0', 'GIERACE_5.0', 'CLINIC_1.0','CLINIC_2.0', 'CLINIC_3.0',  
              'CLINIC_4.0', 'CLINIC_5.0', 'CLINIC_6.0','NFWLKSPD_0.0', 'NFWLKSPD_1.0', 'NFWLKSPD_2.0', 'BUAMEAN', 'QUIMEAN', 'SOSMEAN', 'PHYS_MROS_1.0', 'PHYS_MROS_2.0', 'PHYS_MROS_3.0']

RANDOM_STATE = 43

# fill numerical empty cells with median of the column, race with 1 (for white) and other categorical empty cells with the 0
def fill_empty_cell(sample, attribute, data):
    if pandas.isnull(sample[attribute]):
        if attribute in NUMERICALS:
            return data[attribute].median()
        return int(attribute == 'GIERACE' )
    elif attribute == 'FRAC':
        return int(sample['FAANYSLD'] or sample['FAANYWST'] or sample['FAANYHIP'] or sample['XMDSQGE1'])
    return sample[attribute]


# Hyper parameter tuning
def RandomSearch(estimator, modelName, params, Xtrain, ytrain, Xtest, ytest, score, verb=0):
    t0 = time.time()
    print('\nSearching grid -', modelName, '(' + score + ')...\n')
    cv = KFold(n_splits=10, shuffle=True, random_state=42)
    grid = RandomizedSearchCV(estimator, param_distributions=params, cv=cv, scoring=score, n_jobs=1, verbose=verb)  # n_jobs threads it if possible.
    grid.fit(Xtrain, ytrain)
    numpy.set_printoptions(precision=2)
    return grid.best_estimator_


# calculate confidence interval from the scores
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * numpy.array(data)
    m, se = numpy.mean(a), stats.sem(a)
    h = se * stats.t.ppf((1 + confidence) / 2., len(a) - 1)
    return m, m - h, m + h

# Calculate the 95% CI for AUC in Random Forest
# def conf_auc(test_predictions, ground_truth, bootstrap=1000, seed=None,  confint=0.95):
#     """Takes as input test predictions, ground truth, number of bootstraps, seed, and confidence interval"""
#     #inspired by https://stackoverflow.com/questions/19124239/scikit-learn-roc-curve-with-confidence-intervals by ogrisel
#     bootstrapped_scores = []
#     rng = numpy.random.RandomState(seed)
#     if confint>1:
#         confint=confint/100
#     for i in range(bootstrap):
#         # bootstrap by sampling with replacement on the prediction indices
#         indices = rng.randint(0, len(test_predictions) - 1, len(test_predictions))
#         if len(numpy.unique(ground_truth[indices])) < 2:
#             continue

#         score = metrics.roc_auc_score(ground_truth[indices], test_predictions[indices])
#         bootstrapped_scores.append(score)
    
#     sorted_scores = numpy.array(bootstrapped_scores)
#     sorted_scores.sort()

#     lower_bound=(1-confint)/2
#     upper_bound=1-lower_bound
#     confidence_lower = sorted_scores[int(lower_bound * len(sorted_scores))]
#     confidence_upper = sorted_scores[int(upper_bound * len(sorted_scores))]
#     auc = metrics.roc_auc_score(ground_truth, test_predictions)
#     print("{:0.0f}% confidence interval for the score: [{:0.3f} - {:0.3}] and your AUC is: {:0.3f}".format(confint*100, confidence_lower, confidence_upper, auc))
#     confidence_interval = (confidence_lower, auc, confidence_upper)
#     return confidence_interval

# Calculate the 95% CI for Accuracy in Random Forest
# def conf_accu(test_predictions, ground_truth, bootstrap=1000, seed=None,  confint=0.95):
#     """Takes as input test predictions, ground truth, number of bootstraps, seed, and confidence interval"""
#     #inspired by https://stackoverflow.com/questions/19124239/scikit-learn-roc-curve-with-confidence-intervals by ogrisel
#     bootstrapped_scores = []
#     rng = np.random.RandomState(seed)
#     if confint>1:
#         confint=confint/100
#     for i in range(bootstrap):
#         # bootstrap by sampling with replacement on the prediction indices
#         indices = rng.randint(0, len(test_predictions) - 1, len(test_predictions))
#         if len(np.unique(ground_truth[indices])) < 2:
#             continue
        
#         score = metrics.accuracy_score(ground_truth[indices], test_predictions[indices], normalize=False)
#         bootstrapped_scores.append(score)
    
#     sorted_scores = np.array(bootstrapped_scores)
#     sorted_scores.sort()

#     lower_bound=(1-confint)/2
#     upper_bound=1-lower_bound
#     confidence_lower = sorted_scores[int(lower_bound * len(sorted_scores))]
#     confidence_upper = sorted_scores[int(upper_bound * len(sorted_scores))]
#     auc = metrics.accuracy_score(ground_truth, test_predictions[indices], normalize=False)
#     print("{:0.0f}% confidence interval for the score: [{:0.3f} - {:0.3}] and your accuracy is: {:0.3f}".format(confint*100, confidence_lower, confidence_upper, auc))
#     confidence_interval = (confidence_lower, auc, confidence_upper)
#     return confidence_interval

def get_data(plot=True):
    data = pandas.read_excel('mros_1103snps.xlsx')
    # drop HA_SLDFXFU where only 10% is filled, drop subjectid,
    data.drop(['HA_SLDFXFU', 'TURSMOKE', 'HA_SLDFX', 'HA_WRSTFX'], axis=1, inplace=True)

    # make frac attribute as fracture and return the values from fill_empty_Cell with either FAANYSLD, FAANYWST or HIP
    data['FRAC'] = 0
    for attribute in data.keys():
        data[attribute] = data.apply(lambda sample: fill_empty_cell(sample, attribute, data), axis=1)
    # drop the other fractured values
    data.drop(['FAANYSLD', 'FAANYWST'], axis=1, inplace=True)
    # encode the categorical data
    data = pandas.DataFrame(pandas.get_dummies(data, columns=['GIERACE', 'PHYS_MROS', 'NFWLKSPD']))
    #features = list(data)[20:-6]
    print(data.shape, ' SHAPE OF DAT')
    # setting Y and X
    Y = data['FRAC']
    X = pandas.read_excel('ready_to_go.xlsx')
    #X = data.drop(['SUBJECTID', 'HA_LSD', 'BUAMEAN', 'FAHIPFV1', 'FASLDFV1', 'FAWSTFV1', 'EFSTATUS',
    #                'HA_BMI', 'FAANYHIP', 'HA_CALCIUM', 'XMDSQGE1', 'XMSQGE2', 'CLINIC', 'FRAC'], axis=1)

    #feature_data = data[features]

    #X.drop(features, axis=1, inplace=True)
    # smote
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
    sm = SMOTE(random_state=2, ratio = 1.0)
    
    x_train_s, y_train_s = sm.fit_resample(x_train, y_train)
    # Xtrain, Xtest, ytrain, ytest = train_test_split(X_df, Y_df, test_size=0.2)
    parameters = {
        'n_estimators': [300, 800, 1000, 1200],
        'max_features': ['auto', 'sqrt', 0.2],
        'min_samples_split': [10, 20, 5, 2],
        'max_depth': [2, 3, 5, 8],
        'random_state': [45]
    }
    # split data for parameter sweep
    gbr = RandomForestClassifier()
    model = RandomSearch(estimator=gbr, modelName='Random Forest Classifier', params=parameters, Xtrain=x_train_s, ytrain=y_train_s, Xtest=x_test, ytest=y_test, score='roc_auc')
    # model=RandomForestClassifier(n_estimators=800, max_depth=5)
    model.fit(x_train_s, y_train_s)
#     print(model.feature_importances_)
    # ypred = model.predict(Xtest)
    yscore_raw = model.predict_proba(x_test)
    yscore = [s[1] for s in yscore_raw]
    fpr, tpr, thresh = roc_curve(y_test, yscore)
    print(confusion_matrix(y_test,model.predict(x_test)))
    y_pred = model.fit(x_test, y_test)
    print(list(X))
#    conf_auc(y_pred, y_test)
#    conf_accu(y_pred, y_test)
#    print(confusion_matrix(ytest,yscore))
    auc = roc_auc_score(y_test, yscore)
    ytest = numpy.array(y_test)
    
    rf_final = pd.Series(model.predict(x_test))
    rf_final.to_csv('rf_y_pred.csv')
    # Receiver Operating Characteristic-
    if plot:
        import matplotlib.pyplot as plt
        plt.plot(fpr, tpr, '--k', label='%s ROC (area = %0.2f)' % ('Random Forest', auc))

        plt.plot([0, 1], [0, 1],'-k')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('1-Specificity(False Positive Rate)')
        plt.ylabel('Sensitivity(True Positive Rate)')
        plt.title('Random Forest')
        plt.legend(loc="lower right")
        plt.savefig('random_forest_MOF.png')
        plt.show()
    return fpr, tpr, thresh, auc


if __name__ == '__main__':
    get_data()

In [ ]:
# Print the feature importance
import itertools
import time

import numpy
import pandas
import scikitplot
from imblearn.over_sampling import SMOTE
from matplotlib import pyplot
from scipy import stats
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, brier_score_loss, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold  # http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html#sklearn.model_selection.StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split

# fill numerical empty cells with median of the column, race with 1 (for white) and other categorical empty cells with the 0
def fill_empty_cell(sample, attribute, data):
    if pandas.isnull(sample[attribute]):
        if attribute in NUMERICALS:
            return data[attribute].median()
        return int(attribute == 'GIERACE' )
    elif attribute == 'FRAC':
        return int(sample['FAANYSLD'] or sample['FAANYWST'] or sample['FAANYHIP'])
    return sample[attribute]

NUMERICALS = ['GIAGE1', 'HA_HEIGHT', 'HA_WEIGHT', 'TUDRPRWK', 'B1FND', 'GRS_FN', 'HA_SMOKE_0.0', 'HA_SMOKE_1.0', 'HA_SMOKE_2.0', 'GIERACE_1.0',
              'GIERACE_2.0', 'GIERACE_3.0', 'GIERACE_4.0', 'GIERACE_5.0', 'CLINIC_1.0','CLINIC_2.0', 'CLINIC_3.0',  
              'CLINIC_4.0', 'CLINIC_5.0', 'CLINIC_6.0','NFWLKSPD_0.0', 'NFWLKSPD_1.0', 'NFWLKSPD_2.0']

# Hyper parameter tuning
def RandomSearch(estimator, modelName, params, Xtrain, ytrain, Xtest, ytest, score, verb=0):
    t0 = time.time()
    print('\nSearching grid -', modelName, '(' + score + ')...\n')
    cv = KFold(n_splits=10, shuffle=True, random_state=42)
    grid = RandomizedSearchCV(estimator, param_distributions=params, cv=cv, scoring=score, n_jobs=1, verbose=verb)  # n_jobs threads it if possible.
    grid.fit(Xtrain, ytrain)
    numpy.set_printoptions(precision=2)
    return grid.best_estimator_

data = pandas.read_excel('mros_1103snps.xlsx')
# drop HA_SLDFXFU where only 10% is filled, drop subjectid,
data.drop(['HA_SLDFXFU', 'TURSMOKE', 'HA_SLDFX', 'HA_WRSTFX'], axis=1, inplace=True)

# make frac attribute as fracture and return the values from fill_empty_Cell with either FAANYSLD, FAANYWST or HIP
data['FRAC'] = 0
for attribute in data.keys():
    data[attribute] = data.apply(lambda sample: fill_empty_cell(sample, attribute, data), axis=1)
# drop the other fractured values
data.drop(['FAANYSLD', 'FAANYWST'], axis=1, inplace=True)
# encode the categorical data
data = pandas.DataFrame(pandas.get_dummies(data, columns=['GIERACE', 'PHYS_MROS', 'NFWLKSPD']))
#features = list(data)[20:-6]
print(data.shape, ' SHAPE OF DAT')
# setting Y and X
Y = data['FRAC']
X = pandas.read_excel('ready_to_go.xlsx')
#X = data.drop(['SUBJECTID', 'HA_LSD', 'BUAMEAN', 'FAHIPFV1', 'FASLDFV1', 'FAWSTFV1', 'EFSTATUS',
#                'HA_BMI', 'FAANYHIP', 'HA_CALCIUM', 'XMDSQGE1', 'XMSQGE2', 'CLINIC', 'FRAC'], axis=1)

#feature_data = data[features]

#X.drop(features, axis=1, inplace=True)
# smote
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
sm = SMOTE(random_state=2, ratio = 1.0)
    
x_train_s, y_train_s = sm.fit_resample(x_train, y_train)
# Xtrain, Xtest, ytrain, ytest = train_test_split(X_df, Y_df, test_size=0.2)

parameters = {
        'n_estimators': [300, 800, 1000, 1200],
        'max_features': ['auto', 'sqrt', 0.2],
        'min_samples_split': [10, 20, 5, 2],
        'max_depth': [2, 3, 5, 8],
        'random_state': [42]
    }
# split data for parameter sweep
gbr = RandomForestClassifier()
model = RandomSearch(estimator=gbr, modelName='Random Forest Classifier', params=parameters, Xtrain=x_train_s, ytrain=y_train_s, Xtest=x_test, ytest=y_test, score='roc_auc')
print(model.feature_importances_)
    
feature_impor = pd.Series(model.feature_importances_)
feature_impor.to_csv('feature_impor.csv')    
    

In [ ]:
# Gradient Boosting

import itertools
from datetime import datetime

import numpy
import pandas
import scikitplot
from imblearn.over_sampling import SMOTE
from matplotlib import pyplot
from scipy import stats
from sklearn import preprocessing
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import KFold  # http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html#sklearn.model_selection.StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
try:
    from sksurv.nonparametric import kaplan_meier_estimator
except ImportError:
    kaplan_meier_estimator = None

NUMERICALS = ['GIAGE1', 'HA_HEIGHT', 'HA_WEIGHT', 'TUDRPRWK', 'B1FND', 'HA_SMOKE_0.0', 'HA_SMOKE_1.0', 'HA_SMOKE_2.0', 'GIERACE_1.0',
              'GIERACE_2.0', 'GIERACE_3.0', 'GIERACE_4.0', 'GIERACE_5.0', 'CLINIC_1.0','CLINIC_2.0', 'CLINIC_3.0',  
              'CLINIC_4.0', 'CLINIC_5.0', 'CLINIC_6.0','NFWLKSPD_0.0', 'NFWLKSPD_1.0', 'NFWLKSPD_2.0', 'BUAMEAN', 'QUIMEAN', 'SOSMEAN', 'PHYS_MROS_1.0', 'PHYS_MROS_2.0', 'PHYS_MROS_3.0']

RANDOM_STATE = 42

def fill_empty_cell(sample, attribute, data):
    # numericals = ['B1FND', 'B1THD', 'B1TLD', 'BUAMEAN', 'GIAGE1', 'HA_BMI', 'HA_CALCIUM', 'HA_HEIGHT', 'HA_LSD', 'HA_WEIGHT', 'TUDRPRWK']
    if pandas.isnull(sample[attribute]):
        return data[attribute].median() if attribute in NUMERICALS else int(attribute == 'GIERACE')
    elif attribute == 'FRAC':
        return int(sample['FAANYSLD'] or sample['FAANYWST'] or sample['FAANYHIP'] or sample['XMDSQGE1'])
    elif attribute == 'STATUS':
        return int(sample['EFSTATUS'] > 0) or int(sample['FAANYSLD'] or sample['FAANYWST'] or sample['FAANYHIP'])
    elif attribute == 'DAYS':
        return min(sample['FAHIPFV1'], sample['FASLDFV1'], sample['FAWSTFV1'])
    else:
        return sample[attribute]

# Hyper parameter tuning
def RandomSearch(estimator, modelName, params, Xtrain, ytrain, Xtest, ytest, score, verb=0):
    t0 = datetime.now()
    print('\nSearching grid -', modelName, '(' + score + ')...\n')
    grid = RandomizedSearchCV(estimator, param_distributions=params, cv=KFold(n_splits=3, shuffle=True), scoring=score, n_jobs=1, verbose=verb)  # n_jobs threads it if possible.
    grid.fit(Xtrain, ytrain)
    print('The best parameters are\n', grid.best_params_)
    print('The best ' + score + ' score is %0.4f \n' % (grid.best_score_ * 100))
    ypred = grid.predict(Xtest)
    # cm = confusion_matrix(ytest, ypred)
    numpy.set_printoptions(precision=2)
    # pyplot.figure()
    # plot_confusion_matrix(cm, classes=[0, 1], normalize=False, title='Normalized confusion matrix')
    # pyplot.show()
    print('\n', (datetime.now() - t0).total_seconds(), 'seconds')
    return grid.best_estimator_


# calculate confidence interval from the scores
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * numpy.array(data)
    n = len(a)
    m, se = numpy.mean(a), stats.sem(a)
    h = se * stats.t.ppf((1 + confidence) / 2., n - 1)
    return m, m - h, m + h


def get_data(plot=True):
    data = pandas.read_excel('mros_1103snps.xlsx')
    # drop HA_SLDFXFU where only 10% is filled, drop subjectid,
    data.drop(['HA_SLDFXFU', 'TURSMOKE', 'HA_SLDFX', 'HA_WRSTFX'], axis=1, inplace=True)

    # make frac attribute as fracture and return the values from fill_empty_Cell with either FAANYSLD, FAANYWST or HIP
    data['FRAC'] = 0
    for attribute in data.keys():
        data[attribute] = data.apply(lambda sample: fill_empty_cell(sample, attribute, data), axis=1)
    # drop the other fractured values
    data.drop(['FAANYSLD', 'FAANYWST'], axis=1, inplace=True)
    # encode the categorical data
    data = pandas.DataFrame(pandas.get_dummies(data, columns=['GIERACE', 'PHYS_MROS', 'NFWLKSPD']))
    #features = list(data)[20:-6]
    print(data.shape, ' SHAPE OF Data')
    # setting Y and X
    Y = data['FRAC']
    X = pandas.read_excel('ready_to_go.xlsx')    
    #X = data.drop(['SUBJECTID', 'HA_LSD', 'BUAMEAN', 'FAHIPFV1', 'FASLDFV1', 'FAWSTFV1', 'EFSTATUS',
    #               'HA_BMI', 'FAANYHIP', 'HA_CALCIUM', 'XMDSQGE1', 'XMSQGE2', 'CLINIC', 'FRAC'], axis=1)

    # X_df['GRS_LS'] = feature_data.dot(weight_LS)
    # X_df['GRS_FN'] = feature_data.dot(weight_FN)
    # X.drop(features, axis=1, inplace=True)
    # smote
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
    sm = SMOTE(random_state=2, ratio = 1.0)
    x_train_s, y_train_s = sm.fit_resample(x_train, y_train)
  
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    
    parameters = {
        'loss': ['deviance', 'exponential'],
        'n_estimators': [200, 500, 800, 1000, 1200],
        'learning_rate': [0.001, 0.003, 0.005],
        'subsample': [0.3, 0.5, 0.7, 0.9],  # <1.0 results in reduction of variance and increase in bias
        'min_samples_split': [2, 5, 8],
        'max_features': ['auto', 'log2', 'sqrt', 0.2],
        'random_state': [43],
        'max_depth': [2, 3, 5],
        'min_impurity_decrease': [0.15, 0.1, 0.08, 0.05]
    }
    # split data for parameter sweep
    model = RandomSearch(estimator=GradientBoostingClassifier(), modelName='Gradient Boosting Classifier', params=parameters, Xtrain=x_train_s, ytrain=y_train_s, Xtest=x_test, ytest=y_test, score='roc_auc')
    # model=GradientBoostingClassifier(subsample=0.3, n_estimators=800, min_samples_split=2, min_impurity_decrease=0.05, max_features='sqrt', max_depth=3, loss='deviance', learning_rate=0.01)
    model.fit(x_train_s, y_train_s)
    print(model.feature_importances_)
    # ypred = model.predict(Xtest)
    yscore_raw = model.predict_proba(x_test)
    yscore = [s[1] for s in yscore_raw]
    fpr, tpr, thresh = roc_curve(y_test, yscore)
    auc = roc_auc_score(y_test, yscore)
    ytest = numpy.array(y_test)
    print(confusion_matrix(y_test,model.predict(x_test)))
    
    print(list(X))

    
    gb_final = pd.Series(model.predict(x_test))
    gb_final.to_csv('gb_y_pred.csv')
    
#     gb_y_test = pd.Series(y_test)
#     gb_y_test.to_csv('gb_y_test.csv')
    
    
    # Receiver Operating Characteristic-
    # plot roc curve
    if plot:# Custom settings for the plot
        import matplotlib.pyplot as plt
        plt.plot(fpr, tpr, ',-.k', label='%s ROC (area = %0.2f)' % ('Gradient Boosting', auc))

        plt.plot([0, 1], [0, 1],'-k')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('1-Specificity(False Positive Rate)')
        plt.ylabel('Sensitivity(True Positive Rate)')
        plt.title('Gradient Boosting')
        plt.legend(loc="lower right")
        plt.savefig('gradient_boosting_MOF.png')
        plt.show()   # Display

    return fpr, tpr, thresh, auc


if __name__ == '__main__':
    get_data()



In [ ]:
# MLP

import pickle


with open('datamros1103', 'rb') as file_handler:
    data = pickle.load(file_handler)
    X, Y = data.get('X', []).values, data.get('Y', []).values


import pickle
import numpy as np
import tensorflow as tf
import random as rn

import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE

# fix random seed for reproducibility
seed = 10
#
# # The below is necessary for starting Numpy generated random numbers
# # in a well-defined initial state.
np.random.seed(seed)
# # The below is necessary for starting core Python generated random numbers
# # in a well-defined state.
rn.seed(seed)

# according to keras documentation, numpy seed should be set before importing keras
# information regarding setup for obtaining reproducible results using Keras during development in the following link
# https://keras.io/getting-started/faq/#how-can-i-obtain-reproducible-results-using-keras-during-development
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import label_binarize
from keras.utils import to_categorical

# The below tf.set_random_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
#tf.set_random_seed(seed)
# Y = label_binarize(Y, classes=[0,1])

batch_size = 100
num_classes = 2
optimizer = 'adamax'
from keras import optimizers


number_of_data = X.shape[0]
number_of_train_data = int(.8*number_of_data)
number_of_test_data = number_of_data-number_of_train_data

# load dataset
x_train, x_test = X[:number_of_train_data, :], X[number_of_train_data:, :]
#mean_train_data = np.mean(train_data, axis=0)
#std_train_data = np.std(train_data, axis=0)
#x_train = (train_data - mean_train_data) / std_train_data  # mean variance normalization
#x_test = (test_data - mean_train_data) / std_train_data  # mean variance normalization
y_train, y_test = Y[:number_of_train_data], Y[number_of_train_data:]
#x_test, y_test = sm.fit_resample(X,Y)

# X_df, Xtest, Y_df, ytest = train_test_split(X_df, Y_df, test_size=0.2)
# Xtrain, ytrain = sm.fit_resample(X_df, Y_df)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

y_train = y_train.astype('int32')
y_test = y_test.astype('int32')
# y_train = to_categorical(y_train)
#y_test = to_categorical(y_test)
def create_model():
    model = Sequential()
    model.add(Dense(30, input_dim=26, kernel_initializer='normal', activation='sigmoid'))
    model.add(Dense(20, kernel_initializer='normal', activation='sigmoid'))
    #model.add(Dense(10, kernel_initializer='normal', activation='sigmoid'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer= optimizer, metrics=['accuracy'])
    #model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])
    return model

    # model = Sequential()
    # model.add(Dense(13, input_dim=19, kernel_initializer='normal', activation='relu'))
    # model.add(Dense(6, kernel_initializer='normal', activation='relu'))
    # model.add(Dense(1, kernel_initializer='normal'))
    # # Compile model
    # model.compile(loss='mean_squared_error', optimizer='adam')
    # return model

from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import confusion_matrix
from sklearn import metrics

sm = SMOTE(random_state=2, ratio = 1.0)
x_train_s, y_train_s = sm.fit_resample(x_train, y_train)

model = create_model()
model.fit(x_train_s, y_train_s, epochs=100, batch_size=batch_size, verbose=1)

y_pred = model.predict(x_test)

print(y_pred, ' Predicted Y')

# y_pred = model.predict(x_test)
#y_pred = y_pred.astype('int32')
#print(y_pred, ' YPOST')
yscore_raw = model.predict_proba(x_test)
yscore = [s[0] for s in yscore_raw]
fpr, tpr, thresholds = roc_curve(y_test, yscore)
#final = [(lambda i: 0 if i <= np.average(thresholds) else 1)(i) for i in y_pred]
final = [(lambda i: 0 if i <= np.average(thresholds) else 1)(i) for i in y_pred]
print(confusion_matrix(y_test, final))

mlp_final = pd.Series(final)
mlp_final.to_csv('mlp_y_pred.csv')

auc = metrics.roc_auc_score(y_test, yscore)
plt.plot(fpr, tpr, ':k', label='%s ROC (area = %0.2f)' % ('Neural Network', auc))

# Calculate Area under the curve to display on the plot

# Now, plot the computed values

# Custom settings for the plot
# Receiver Operating Characteristic-
plt.plot([0, 1], [0, 1],'-k')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1-Specificity(False Positive Rate)')
plt.ylabel('Sensitivity(True Positive Rate)')
plt.title('Neural Network')
plt.legend(loc="lower right")
plt.savefig('mlp_MOF.png')
plt.show()   # Display